In [22]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
# Set the paths to your training and validation data directories
train_data_dir = '../SFSU_Art/training'
val_data_dir = '../SFSU_Art/validation'

# Set the desired image size and batch size
image_size = (224, 224)  # Adjust according to EfficientNetB0 input size requirements
batch_size = 10
num_classes = 7  # Set the number of classes in your classification task
print("done")

done


In [24]:
# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
print("done")

done


In [25]:
# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
print("done")

done


In [26]:
# Create the final model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("done")

done


In [27]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data augmentation for validation set (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)
print("done")

done


In [28]:
# Generate batches of augmented training and validation data
train_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
print("done")

Found 206 images belonging to 7 classes.
Found 70 images belonging to 7 classes.
done


In [29]:
# Train the model
epochs = 10
model.fit(train_data, validation_data=val_data, epochs=epochs)
print("done")

Epoch 1/10


2023-05-17 21:39:33.496204: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2023-05-17 21:39:33.565475: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2023-05-17 21:39:33.581387: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2023-05-17 21:39:33.604232: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2023-05-17 21:39:33.616334: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.


21/21 [==============================] - 32s 1s/step - loss: 1.9418 - accuracy: 0.2039 - val_loss: 2.0203 - val_accuracy: 0.1429
Epoch 2/10
21/21 [==============================] - 23s 1s/step - loss: 1.8810 - accuracy: 0.2476 - val_loss: 2.0525 - val_accuracy: 0.1429
Epoch 3/10
21/21 [==============================] - 24s 1s/step - loss: 1.9141 - accuracy: 0.2039 - val_loss: 2.0312 - val_accuracy: 0.1429
Epoch 4/10
21/21 [==============================] - 23s 1s/step - loss: 1.8867 - accuracy: 0.2379 - val_loss: 2.0414 - val_accuracy: 0.1429
Epoch 5/10
21/21 [==============================] - 23s 1s/step - loss: 1.8544 - accuracy: 0.2233 - val_loss: 2.0226 - val_accuracy: 0.1429
Epoch 6/10
21/21 [==============================] - 23s 1s/step - loss: 1.8858 - accuracy: 0.2136 - val_loss: 2.0243 - val_accuracy: 0.1429
Epoch 7/10
21/21 [==============================] - 24s 1s/step - loss: 1.8770 - accuracy: 0.1942 - val_loss: 2.0346 - val_accuracy: 0.1429
Epoch 8/10
21/21 [=============

In [31]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_4 (Rescaling)        (None, None, None,   0           ['input_5[0][0]']                
                                3)                                                                
                                                                                                  
 normalization_4 (Normalization  (None, None, None,   7          ['rescaling_4[0][0]']            
 )                              3)                                                          

                                                                                                  
 block2a_project_conv (Conv2D)  (None, None, None,   2304        ['block2a_se_excite[0][0]']      
                                24)                                                               
                                                                                                  
 block2a_project_bn (BatchNorma  (None, None, None,   96         ['block2a_project_conv[0][0]']   
 lization)                      24)                                                               
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, None, None,   3456        ['block2a_project_bn[0][0]']     
                                144)                                                              
                                                                                                  
 block2b_e

                                40)                                                               
                                                                                                  
 block3a_project_bn (BatchNorma  (None, None, None,   160        ['block3a_project_conv[0][0]']   
 lization)                      40)                                                               
                                                                                                  
 block3b_expand_conv (Conv2D)   (None, None, None,   9600        ['block3a_project_bn[0][0]']     
                                240)                                                              
                                                                                                  
 block3b_expand_bn (BatchNormal  (None, None, None,   960        ['block3b_expand_conv[0][0]']    
 ization)                       240)                                                              
          

 block4a_project_bn (BatchNorma  (None, None, None,   320        ['block4a_project_conv[0][0]']   
 lization)                      80)                                                               
                                                                                                  
 block4b_expand_conv (Conv2D)   (None, None, None,   38400       ['block4a_project_bn[0][0]']     
                                480)                                                              
                                                                                                  
 block4b_expand_bn (BatchNormal  (None, None, None,   1920       ['block4b_expand_conv[0][0]']    
 ization)                       480)                                                              
                                                                                                  
 block4b_expand_activation (Act  (None, None, None,   0          ['block4b_expand_bn[0][0]']      
 ivation) 

                                                                                                  
 block4c_add (Add)              (None, None, None,   0           ['block4c_drop[0][0]',           
                                80)                               'block4b_add[0][0]']            
                                                                                                  
 block5a_expand_conv (Conv2D)   (None, None, None,   38400       ['block4c_add[0][0]']            
                                480)                                                              
                                                                                                  
 block5a_expand_bn (BatchNormal  (None, None, None,   1920       ['block5a_expand_conv[0][0]']    
 ization)                       480)                                                              
                                                                                                  
 block5a_e

                                672)                                                              
                                                                                                  
 block5c_expand_bn (BatchNormal  (None, None, None,   2688       ['block5c_expand_conv[0][0]']    
 ization)                       672)                                                              
                                                                                                  
 block5c_expand_activation (Act  (None, None, None,   0          ['block5c_expand_bn[0][0]']      
 ivation)                       672)                                                              
                                                                                                  
 block5c_dwconv (DepthwiseConv2  (None, None, None,   16800      ['block5c_expand_activation[0][0]
 D)                             672)                             ']                               
          

 block6b_expand_bn (BatchNormal  (None, None, None,   4608       ['block6b_expand_conv[0][0]']    
 ization)                       1152)                                                             
                                                                                                  
 block6b_expand_activation (Act  (None, None, None,   0          ['block6b_expand_bn[0][0]']      
 ivation)                       1152)                                                             
                                                                                                  
 block6b_dwconv (DepthwiseConv2  (None, None, None,   28800      ['block6b_expand_activation[0][0]
 D)                             1152)                            ']                               
                                                                                                  
 block6b_bn (BatchNormalization  (None, None, None,   4608       ['block6b_dwconv[0][0]']         
 )        

                                                                                                  
 block6d_expand_bn (BatchNormal  (None, None, None,   4608       ['block6d_expand_conv[0][0]']    
 ization)                       1152)                                                             
                                                                                                  
 block6d_expand_activation (Act  (None, None, None,   0          ['block6d_expand_bn[0][0]']      
 ivation)                       1152)                                                             
                                                                                                  
 block6d_dwconv (DepthwiseConv2  (None, None, None,   28800      ['block6d_expand_activation[0][0]
 D)                             1152)                            ']                               
                                                                                                  
 block6d_b

                                1280)                                                             
                                                                                                  
 global_average_pooling2d_2 (Gl  (None, 1280)        0           ['top_activation[0][0]']         
 obalAveragePooling2D)                                                                            
                                                                                                  
 dense_3 (Dense)                (None, 256)          327936      ['global_average_pooling2d_2[0][0
                                                                 ]']                              
                                                                                                  
 dense_4 (Dense)                (None, 7)            1799        ['dense_3[0][0]']                
                                                                                                  
Total para

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import PIL

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('Efficient.tflite', 'wb') as f:
  f.write(tflite_model)

print("done")